In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

C:\Users\Andrew\AppData\Local\Temp\ipykernel_19944\1222050232.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [14]:
def get_reviews(appid, params={'json':1}):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+appid, params=params)
        return response.json()

In [13]:
get_reviews('1593500')

{'success': 1,
 'query_summary': {'num_reviews': 20,
  'review_score': 9,
  'review_score_desc': 'Overwhelmingly Positive',
  'total_positive': 42630,
  'total_negative': 1216,
  'total_reviews': 43846},
 'reviews': [{'recommendationid': '158566865',
   'author': {'steamid': '76561199337484291',
    'num_games_owned': 16,
    'num_reviews': 7,
    'playtime_forever': 663,
    'playtime_last_two_weeks': 269,
    'playtime_at_review': 220,
    'last_played': 1709636526},
   'language': 'english',
   'review': "A damn good game demonstrating a beautiful relationship between a father and his son. Kratos is not only a loving father, he teaches son of war how to fight adding a new element of love into the new god of war series, which i love.\nAlso my friend bet $50 i won't get 10 people who found this helpful, so help a man out would you?\n",
   'timestamp_created': 1708206529,
   'timestamp_updated': 1708206612,
   'voted_up': True,
   'votes_up': 98,
   'votes_funny': 2,
   'weighted_vote_

In [21]:
def get_n_reviews(appid, n=100):
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'language' : 'english',
            'day_range' : 365,
            'purchase_type' : 'all',
            'num_per_page' : '100'
            }
    while True:
        params['cursor'] = cursor.encode()
        params['num_per_page'] = n

        response = get_reviews(appid, params) # Getting reviews 
        cursor = response['cursor'] # The next cursor value for the next batch
        reviews += response['reviews'] # Appending reviews

        if len(reviews) == 10000:break

    return reviews

In [22]:
x = get_n_reviews('1593500')


KeyboardInterrupt



In [20]:
len(x)

1000

In [8]:
x[['review', 'voted_up']]

TypeError: list indices must be integers or slices, not list

In [5]:
def get_n_appids(n=100, filter_by='topsellers'):
    appids = []
    url = f'https://store.steampowered.com/search/?category1=998&filter={filter_by}&page='
    page = 0
    while page*25 < n:
        page += 1
        response = requests.get(url=url+str(page))
        soup = BeautifulSoup(response.text, 'html.parser')
        for row in soup.find_all(class_='search_result_row'):
            appids.append(row['data-ds-appid'])
    return appids[:n]

In [6]:
ids = get_n_appids(n=1)

In [7]:
ids

['553850']

In [9]:
reviews = []
appids = get_n_appids(100)
for appid in appids:
    reviews += get_n_reviews(appid, 100)
df = pd.DataFrame(reviews)[['review', 'voted_up']]
df

,review,voted_up
0,I saw the hype online and pretty much ignored ...,True
1,"Crazy to say that in February 2024, a successf...",True
2,The only co-op game where you are scared of yo...,True
3,"Worked so hard to save Super-Earth, only to be...",True
4,"Thank you Jimmy525521,\nI will never forget th...",True
...,...,...
9445,"Pros:\n-Amazing Combat, seriously cannot be ov...",True
9446,You can climb on a Dragon and stab it. ‘Nough ...,True
9447,Dragon's Dogma was a game that came out just a...,True
9448,---{ Graphics }---\n☐ You forget what reality ...,True


In [12]:
df.to_csv("Reviews.csv")

In [14]:
pd.read_csv("Reviews.csv")

,Unnamed: 0,review,voted_up
0,0,I saw the hype online and pretty much ignored ...,True
1,1,"Crazy to say that in February 2024, a successf...",True
2,2,The only co-op game where you are scared of yo...,True
3,3,"Worked so hard to save Super-Earth, only to be...",True
4,4,"Thank you Jimmy525521,\nI will never forget th...",True
...,...,...,...
9445,9445,"Pros:\n-Amazing Combat, seriously cannot be ov...",True
9446,9446,You can climb on a Dragon and stab it. ‘Nough ...,True
9447,9447,Dragon's Dogma was a game that came out just a...,True
9448,9448,---{ Graphics }---\n☐ You forget what reality ...,True
